In [ ]:
# for data processing
import pandas as pd
import io
import re

# for send request
import requests

# for displaying a progress bar
from tqdm.notebook import tqdm

# for user input for your Google Api Key
from getpass import getpass

# for downloading csv file(if you use Google Colab)
from google.colab import files

In [ ]:
# upload tokyo hotel list csv
tokyo_hotel_csv = files.upload()

Saving hotel_list.csv to hotel_list.csv


In [ ]:
# create a new DataFrame from the csv
hotel_df = pd.read_csv(io.BytesIO(tokyo_hotel_csv['hotel_list.csv']), index_col=0)
hotel_df = hotel_df[hotel_df['hotel_price($)'] >= 10]
hotel_df.head()

,hotel_name,hotel_type,hotel_lat,hotel_lon,hotel_price($)
0,東京秋葉原菲諾貝斯特韋斯特飯店 (Best Western Hotel Fino Tokyo...,Hotel,35.703177,139.775658,650
7,青年旅館澀谷芥末飯店 - 青年旅舍 (MUSTARD HOTEL SHIBUYA - Hos...,Hostel,35.653490,139.707100,803
14,HOTEL MYSTAYS 淺草橋 (Hotel MyStays Asakusa-bashi),Hotel,35.697162,139.783486,791
32,羽田放鬆頂級飯店 (RELIEF PREMIUM Haneda by RELIEF),Hotel,35.557167,139.725076,915
51,貝斯特飯店 (Best Hotel),Hotel,35.699171,139.701386,787


In [ ]:
# input the Google API key
GOOGLE_API_KEY = getpass('INPUT YOUR GOOGLE API KEY:')

INPUT YOUR GOOGLE API KEY:··········


In [ ]:
# get hotel area and address
hotel_value_list = []
outlier_list = []
row_count = hotel_df.shape[0]
j=0
for i in tqdm(range(row_count)):
  row = hotel_df.iloc[i]
  hotel_name = row[0]
  hotel_type = row[1]
  hotel_lat = row[2]
  hotel_lon = row[3]
  hotel_price = row[4]
  
  link = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={},{}&key={}'.format(hotel_lat, hotel_lon, GOOGLE_API_KEY)
  r = requests.get(link)
  map_json = r.json()
  if map_json['status'] != 'OK':
    hotel_address, hotel_area = None, None
  else:
    for area in map_json['results'][0]['address_components']:
      if 'locality' in area['types']:
        hotel_area = area['long_name']
    hotel_address = map_json['results'][0]['formatted_address']
    if 'City' not in hotel_area and hotel_area not in outlier_list:
      if re.match(r'[A-Z]', hotel_area) != None:
        continue
      else:
        outlier_list.append(hotel_area)
    
  hotel_value_list.append([hotel_name, hotel_type, hotel_address, hotel_area, hotel_lat, hotel_lon, hotel_price])

In [ ]:
# create a new DataFrame
columns_list = ['hotel_name', 'hotel_type', 'hotel_address', 'hotel_area', 'hotel_lat', 'hotel_lon', 'hotel_price']
hotel_df_mod = pd.DataFrame(hotel_value_list, columns=columns_list)
hotel_df_mod.head()

,hotel_name,hotel_type,hotel_address,hotel_area,hotel_lat,hotel_lon,hotel_price
0,東京秋葉原菲諾貝斯特韋斯特飯店 (Best Western Hotel Fino Tokyo...,Hotel,"5-chōme-1-6 Ueno, Taito City, Tōkyō-to 110-000...",Taito City,35.703177,139.775658,650
1,青年旅館澀谷芥末飯店 - 青年旅舍 (MUSTARD HOTEL SHIBUYA - Hos...,Hostel,"1-chōme-29-3 Higashi, Shibuya City, Tōkyō-to 1...",Shibuya City,35.653490,139.707100,803
2,HOTEL MYSTAYS 淺草橋 (Hotel MyStays Asakusa-bashi),Hotel,"Japan, 〒111-0053 Tōkyō-to, Taito City, Asakusa...",Taito City,35.697162,139.783486,791
3,羽田放鬆頂級飯店 (RELIEF PREMIUM Haneda by RELIEF),Hotel,"1-chōme-25-13 Minamikamata, Ota City, Tōkyō-to...",Ota City,35.557167,139.725076,915
4,貝斯特飯店 (Best Hotel),Hotel,"1-chōme-5-21 Hyakuninchō, Shinjuku City, Tōkyō...",Shinjuku City,35.699171,139.701386,787


In [ ]:
# check outliers that should be fixed
print(outlier_list)

[]


In [ ]:
# replace values
replace_dic = {'中央区':'Chuo City', '渋谷区':'Shibuya City', '新宿区':'Shinjuku City', 
               '港区':'Minato City', '台東区':'Daito City', '千代田区':'Chiyoda City'}
hotel_df_mod.replace(replace_dic, inplace=True)
hotel_df_mod['hotel_area'].unique()

array(['Taito City', 'Shibuya City', 'Ota City', 'Shinjuku City',
       'Arakawa City', 'Sumida City', 'Koto City', 'Suginami City',
       'Chiyoda City', 'Minato City', 'Toshima City', 'Chuo City',
       'Meguro City', 'Itabashi City', 'Katsushika City', 'Kita City',
       'Setagaya City', 'Nakano City'], dtype=object)

In [ ]:
# save the DataFrame as a new CSV file
hotel_df_mod.to_csv('tokyo_hotel_list_complete.csv')
files.download('tokyo_hotel_list_complete.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>